# Prediction Model Blueprint

Thema: Energie-Verbrauchs-Prognose

Autorin: Ananya W.

Pakete: [pandas](https://pandas.pydata.org/docs/reference/index.html), [scikit-learn (sklearn)](https://scikit-learn.org/stable/api/index.html), [seaborn](https://seaborn.pydata.org/tutorial.html)

Dateien:`blueprint_data_{train/assessment}.csv`

Die Arbeiten von Ananya in diesem Dokument gliedern sich in 5 Phasen (Schritte):
1. Datenanalyse
2. Datenbereinigung
3. Feature Engineering
4. Datensatz-Splitting
5. Modell-Training und -Test
6. Evaluation und Kapselung der Ergebnisse
   
Außerdem hat Ihnen Ananya ganz am Ende des Notebooks noch eine Nachricht hinterlassen.

## 1. Datenanalyse
Zunächst fokussieren wir uns auf eine kurze explorative Datenanalyse, um einen ersten Überblick zu erhalten. 

### Einlesen des Datensatzes

Einlesen des Datensatzes und Ausgabe von Dimension, Spalten und Statistiken.

In [ ]:
import pandas as pd

dataframe = pd.read_csv('blueprint_data_train.csv')

Ausgabe grundlegender Informationen zum Datensatz.

In [ ]:
dataframe.info()

Die Spalte `energy_joule` enthält null-Werte die behandelt werden müssen. Das wird in Teil 2 (Datenbereinigung) behandelt.

Verwenden der Methode `describe()` zur Ausgabe der Wertebereiche und einfacher Statistiken der Spalten:

In [ ]:
dataframe.describe()

Die Spalte `location_id` hat Varianz 0 (alle Einträge haben den Wert 1). Die Spalte wird in Teil 2 (Datenbereinigung) entfernt.

Ausgabe der ersten Zeilen des Datensatzes:

In [ ]:
dataframe.head(10)

### Erstellung einfacher Visualisierungen

Zur Visualisierung wird die externe Bibliothek `seaborn` verwendet.

Anzeige von Histogrammen für jede Spalte.

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

for col in dataframe.columns:
    print(col)
    plt.figure(1, figsize=(5,5))
    sns.histplot(dataframe, x=col, discrete=True)
    plt.show()

Das dritte Histogramm zeigt, dass es zwei Arten von Robotern gibt, nämlich „Robot1“ und „Robot2“. Diese sind als Zeichenketten (strings) kodiert und müssen als numerische Werte kodiert werden, da die meisten Modelle für maschinelles Lernen nur numerische Eingaben akzeptieren. Dies wird in Teil 3 (Feature Engineering) behandelt.

Paarweise Visualisierung der Verteilungen zweier Spalten.

Hierbei ist insbesondere die Zeile zum Vorhersageziel `energy_joule` relevant, um eine Einschätzung abzuleiten, welche Features des Energieverbrauch beeinflussen.

In [ ]:
sns.pairplot(dataframe)

Die Feature-Spalten `distance`, `number_of_turns`, `additional_cargo` und `rush_level` scheinen mit dem Energieverbrauch zu korrelieren und werden daher beibehalten (als Input für das Regressions-Modell).

## 2. Datenbereinigung

In diesem Teil, Behandeln wir zuerst die null-Werte, die die Spalte`energy_joule` enthält. Wir entscheiden uns hier für einfaches Löschen der entsprechenden Zeilen:

In [ ]:
dataframe = dataframe.dropna()

In [ ]:
dataframe.info()

Die Einträge der Spalte `commissioner` sind nicht relevant für den Energieverbrauch des Roboters. 
Den Verdacht legt das logische Verständnis des Problems nahe und er bestätigt sich durch die obige Visualisierung.

Außerdem, hat die Spalte `localtion_id` keine Varianz.

Die Spalten werden aus dem Datensatz entfernt: 

In [ ]:
dataframe = dataframe.drop(columns=["commissioner", "location_id"])

## 3. Feature Engineering

Wie in Teil 1 beobachtet, beschreibt die Spalte `robot` den Typ des Roboters (`Robot1` oder `Robot2`) in String-Format. Die Einträge dieser Spalte können ordinal als Integers encodiert. `Robot1` wird mit `1` ersetzt und `Robot2` wird mit `2` ersetzt:

In [ ]:
def convert_string_robot_to_numeric(str_robot):
    if str_robot == "Robot1":
        return 1
    elif str_robot == "Robot2":
        return 2
    else:
        print("str_robot must be either Robot1 or Robot2")
        print("Returning None")
        return None
        
dataframe["robot"] = dataframe["robot"].map(convert_string_robot_to_numeric)

In [ ]:
display(dataframe)

Man kann statt der durchgeführten ordinalen Encodierung auch binäre Variablen erstellen, die sich auf `Robot1` und `Robot2` beziehen. Das nennt man One-Hot Encoding.

## 4. Datensatz-Splitting

Um die Ergebnisse eines Modells sauber evaluieren zu können, teilen wir unsere Trainingsdaten zunächst in zwei Teile auf:
Der erste Teil wird tatsächlich für das Modell-Training verwendet, der zweite Teil wird für die Evaluation (Test) des Modells vorbehalten.

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(dataframe, test_size=0.3, random_state=23)

## 5. Modell-Training und -Test

Wir trainieren verschiedene Modelle auf dem Trainings-Datensatz und evaluieren diese mit Hilfe der abgespaltenen Testdaten.

Die folgenden Modelle wollen wir betrachten:

* Lineare Regression
* Polynomiale Regression
* Decision Tree
* Random Forest

Lineare und Polynomiale Regression nehmen dabei monotone Zusammenhänge zwischen den Features und dem vorherzusagenden Wert an. 
Wir müssen also bei jedem Input überlegen, ob dieses Feature diese Anforderung erfüllt oder ggf. entsprechend vorverarbeitet werden kann.
Hier hilft Ihnen auch die Dokumentation von Scikit-Learn zu [Linearer Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) und [Polynomialen Features](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).
Der [Scikit-Learn User Guide](https://scikit-learn.org/stable/modules/linear_model.html) liefert hier noch zusätzliche Hinweise. 

Baum-basierte Modelle (z.B. Decision Tree und Random Forest) können in der Regel mit den meisten Features in Rohform gut arbeiten und sind, im Gegensatz zu anderen Verfahren, nicht auf (annähernd) einheitliche Skalierungen oder Varianzen angewiesen.
Einzelne Entscheidungsbäume (Decision Tree) werden in der Praxis selten als Prognosemodell eingesetzt, da Sie oft nicht sehr gut generalisieren, d.h. auf ungesehenen Daten oft keine sehr guten Ergebnisse erzielen.
Stattdessen verwendet man oft sogenannte Random Forests, welche aus sehr vielen, auf Datensatzteilen trainierten Bäumen bestehen, die in gleichen Teilen zum Vorhersage-Ergebnis beitragen.
Nähere Informationen hierzu können dem [Scikit-Learn User Guide](https://scikit-learn.org/stable/modules/tree.html#) und der [Dokumentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) entnommen werden.

Wir trainieren und evaluieren alle vier Modelltypen auf den vorbereiteten Daten und wählen anschließend das beste Modell aus.

In [ ]:
target_column = "energy_joule"
feature_columns = dataframe.columns.drop(target_column)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

mse_dict = {}

##### 1) Lineare Regression

In [ ]:
from sklearn.linear_model import LinearRegression

model_lin_reg = LinearRegression()
model_lin_reg.fit(train_df[feature_columns], train_df[target_column])

predictions = model_lin_reg.predict(test_df[feature_columns])

lin_reg_mse = mean_squared_error(test_df[target_column], predictions)

mse_dict["Linear Regression"] = lin_reg_mse
print(f"MSE: {lin_reg_mse}")

##### 2) Polynomiale Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

preprocessor_pol_reg = PolynomialFeatures(degree=2)
train_features_poly = preprocessor_pol_reg.fit_transform(train_df[feature_columns])
model_pol_reg = LinearRegression()
model_pol_reg.fit(train_features_poly, train_df[target_column])

test_features_poly = preprocessor_pol_reg.transform(test_df[feature_columns])
predictions = model_pol_reg.predict(test_features_poly)
pol_reg_mse = mean_squared_error(test_df[target_column], predictions)

mse_dict["Polynomial Regression"] = pol_reg_mse
print(f"MSE: {pol_reg_mse}")

##### 3) Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model_tree = DecisionTreeRegressor()
model_tree.fit(train_df[feature_columns], train_df[target_column])

predictions = model_tree.predict(test_df[feature_columns])
tree_mse = mean_squared_error(test_df[target_column], predictions)

mse_dict["Decision Tree"] = tree_mse
print(f"MSE: {tree_mse}")

##### 4) Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model_forest = RandomForestRegressor(n_estimators=100)
model_forest.fit(train_df[feature_columns], train_df[target_column])

predictions = model_forest.predict(test_df[feature_columns])
forest_mse = mean_squared_error(test_df[target_column], predictions)

mse_dict["Random Forest"] = forest_mse
print(f"MSE: {forest_mse}")

In [ ]:
for key, value in mse_dict.items():
    print(f"{key:25s}\t{value:>8.3f}")

Die besten Ergebnisse (geringster MSE) wurde mit den Modell der polynomialen Regression erzielt. Dieses wird für die weitere Verwendung empfohlen.

## 6. Evaluation und Kapselung der Ergebnisse
#### Erstellung einer Funktion zur Evaluation eines ungesehenen Datensatzes

Wir implementieren nun eine Funktion, welche das Modell auf ungesehenen Datensätzes in identischem Format ausführt und evaluiert.
Die besten Ergebnisse wurden mit der polynomialen Regression erzielt, daher wird dieses Modell in der Funktion verwendet.

Die Funktion wird anschließend auf dem vorhandenen Test-Datensatz `'blueprint_data_assessment.csv'` ausgeführt und die Ergebnisse angezeigt. 

Zunächst werden die oben durchgeführten Datenvorverarbeitungs-Schritte in einer Funktion zusammengefasst:

In [ ]:
def preprocess_data(df):
    df = df.dropna()
    df = df.drop(columns=["location_id", "commissioner"])
    df["robot"] = df["robot"].map(convert_string_robot_to_numeric) 
    
    return df

Anschließend schreiben wir eine Funktion zur Ausführung des zuvor trainierten Modells auf einem vorverarbeiteten Datensatz:

In [ ]:
def run_model(df):
    features = preprocessor_pol_reg.transform(df[feature_columns])
    predictions = model_pol_reg.predict(features)
    return predictions

Wir fassen nun das Einlesen, die Vorverarbeitung, die Prädiktion, sowie die Berechnung des Fehlers in einer einzigen Funktion zusammen.

In [ ]:
def run_and_evaluate_on_dataset(dataset_path):
    dataset_df = pd.read_csv(dataset_path)
    dataset_proc = preprocess_data(dataset_df)
    dataset_pred = run_model(dataset_proc)
    
    error_mse = mean_squared_error(dataset_proc[target_column], dataset_pred)
    error_mape = mean_absolute_percentage_error(dataset_proc[target_column], dataset_pred)
    
    return error_mse, error_mape

Wir führen die oben genannten Schritte auf dem Testdatensatz mit Hilfe eines einzelnen Funktionsaufrufes durch und geben die Ergebnisse auf der Konsole aus.

In [ ]:
file = 'blueprint_data_assessment.csv'
error_mse, error_mape = run_and_evaluate_on_dataset(file)
print(f"Evaluationsergebnisse der Daten {file}")
print(f"{'Error MSE':30s} {error_mse:>10.3f}")
print(f"{'Error MAPE':30s} {100*error_mape:>10.2f} %")

Es wird eine mittlere prozentuale Abweichung der Vorhersage des Energieverbrauchs von 5,17 % auf dem assessment-Datensatz erreicht. Ein solides Ergebnis!

### Eine Nachricht von Ananya:

Wir sind total gespannt wie der lange versprochene Datensatz mit den Aufzeichnungen des Produktiv-Systems aussehen wird!
Hoffentlich können unsere Ansätze und Modelle auch für diese neuen Daten verwendet werden.

Leider werden wir nicht mehr im Projektteam sein, um den neuen Datensatz selbst zu analysieren.
Wer auch immer das Projekt dann bearbeiten mag - wir hoffen, unsere bereits durchgeführten Analysen und Experimente helfen unseren Nachfolgern bei der Lösung dieses spannenden Problems.


Alles Gute und viel Erfolg!

Ananya W.   